In [158]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import json
%matplotlib inline
import pandas as pd
from sklearn.linear_model import LogisticRegression

pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 300

## Predicting "Greenness" Of Content

This dataset comes from [stumbleupon](https://www.stumbleupon.com/), a web page recommender and was made available [here](https://www.kaggle.com/c/stumbleupon/download/train.tsv)

A description of the columns is below

FieldName|Type|Description
---------|----|-----------
url|string|Url of the webpage to be classified
urlid|integer| StumbleUpon's unique identifier for each url
boilerplate|json|Boilerplate text
alchemy_category|string|Alchemy category (per the publicly available Alchemy API found at www.alchemyapi.com)
alchemy_category_score|double|Alchemy category score (per the publicly available Alchemy API found at www.alchemyapi.com)
avglinksize| double|Average number of words in each link
commonLinkRatio_1|double|# of links sharing at least 1 word with 1 other links / # of links
commonLinkRatio_2|double|# of links sharing at least 1 word with 2 other links / # of links
commonLinkRatio_3|double|# of links sharing at least 1 word with 3 other links / # of links
commonLinkRatio_4|double|# of links sharing at least 1 word with 4 other links / # of links
compression_ratio|double|Compression achieved on this page via gzip (measure of redundancy)
embed_ratio|double|Count of number of <embed> usage
frameBased|integer (0 or 1)|A page is frame-based (1) if it has no body markup but have a frameset markup
frameTagRatio|double|Ratio of iframe markups over total number of markups
hasDomainLink|integer (0 or 1)|True (1) if it contains an <a> with an url with domain
html_ratio|double|Ratio of tags vs text in the page
image_ratio|double|Ratio of <img> tags vs text in the page
is_news|integer (0 or 1) | True (1) if StumbleUpon's news classifier determines that this webpage is news
lengthyLinkDomain| integer (0 or 1)|True (1) if at least 3 <a> 's text contains more than 30 alphanumeric characters
linkwordscore|double|Percentage of words on the page that are in hyperlink's text
news_front_page| integer (0 or 1)|True (1) if StumbleUpon's news classifier determines that this webpage is front-page news
non_markup_alphanum_characters|integer| Page's text's number of alphanumeric characters
numberOfLinks|integer Number of <a>|markups
numwords_in_url| double|Number of words in url
parametrizedLinkRatio|double|A link is parametrized if it's url contains parameters or has an attached onClick event
spelling_errors_ratio|double|Ratio of words not found in wiki (considered to be a spelling mistake)
label|integer (0 or 1)|User-determined label. Either evergreen (1) or non-evergreen (0); available for train.tsv only

### What are 'evergreen' sites?
- These are websites that always relevant like recipes or reviews (as opposed to current events)
- Look at some examples

In [159]:
import os
os.getcwd()


data = pd.read_csv('data/train.tsv', sep='\t', na_values={'is_news' : '?'}).fillna(0)

# Extract the title and body from the boilerplate JSON text
data['title'] = data.boilerplate.map(lambda x: json.loads(x).get('title', ''))
data['body'] = data.boilerplate.map(lambda x: json.loads(x).get('body', ''))

In [160]:
data.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title,body
0,http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html,4042,"{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,0.443783,0.0,0,0.090774,0,0.245831,0.003883,1.0,1,24,0,5424,170,8,0.152941,0.079130,0,"IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries",A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose Califor...
1,http://www.popsci.com/technology/article/2012-07/electronic-futuristic-starting-gun-eliminates-advantages-races,8471,"{""title"":""The Fully Electronic Futuristic Starting Gun That Eliminates Advantages in Races the fully electronic, futuristic starting gun that eliminates advantages in races the fully electronic, futuristic starting gun that eliminates advantages in races"",""body"":""And that can be carried on a pla...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,0.468649,0.0,0,0.098707,0,0.203490,0.088652,1.0,1,40,0,4973,187,9,0.181818,0.125448,1,"The Fully Electronic Futuristic Starting Gun That Eliminates Advantages in Races the fully electronic, futuristic starting gun that eliminates advantages in races the fully electronic, futuristic starting gun that eliminates advantages in races",And that can be carried on a plane without the hassle too The Omega E Gun Starting Pistol Omega It s easy to take for granted just how insanely close some Olympic races are and how much the minutiae of it all can matter The perfect example is the traditional starting gun Seems easy You pull a tr...
2,http://www.menshealth.com/health/flu-fighting-fruits?cm_mmc=Facebook-_-MensHealth-_-Content-Health-_-FightFluWithFruit,1164,"{""title"":""Fruits that Fight the Flu fruits that fight the flu | cold & flu | men's health"",""body"":""Apples The most popular source of antioxidants in our diet one apple has an antioxidant effect equivalent to 1 500 mg of vitamin C Apples are loaded with protective flavonoids which may prevent hea...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,0.525448,0.0,0,0.072448,0,0.226402,0.120536,1.0,1,55,0,2240,258,11,0.166667,0.057613,1,Fruits that Fight the Flu fruits that fight the flu | cold & flu | men's health,Apples The most popular source of antioxidants in our diet one apple has an antioxidant effect equivalent to 1 500 mg of vitamin C Apples are loaded with protective flavonoids which may prevent heart disease and cancer Next Papayas With 250 percent of the RDA of vitamin C a papaya can help kick ...
3,http://www.dumblittleman.com/2007/12/10-foolproof-tips-for-better-sleep.html,6684,"{""title"":""10 Foolproof Tips for Better Sleep "",""body"":""There was a period in my life when I had a lot of problems with sleep It took me very long to fall asleep I was easily awaken and I simply wasn t getting enough of rest at night I didn t want to take medication and this led me to learn sever...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,0.480725,0.0,0,0.095861,0,0.265656,0.035343,1.0,0,24,0,2737,1

#### Does being a news site effect green-ness?

In [161]:
data[['news_front_page']]

,news_front_page
0,0
1,0
2,0
3,0
4,0
5,?
6,0
7,?
8,0
9,?


In [163]:
#muther fucking data has ?. I want to get rid of these
# ?str.replace(old, new[, max])
# data.news_front_page.str.replace('?',"Nan").astype(float)

good_news = data[(data['news_front_page'] != "?")]
good_news.describe()

,urlid,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,is_news,lengthyLinkDomain,linkwordscore,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
count,6147.000000,6147.000000,6147.000000,6147.000000,6147.000000,6147.000000,6147.000000,6147.000000,6147.0,6147.000000,6147.000000,6147.000000,6147.000000,6147.000000,6147.000000,6147.000000,6147.000000,6147.000000,6147.00000,6147.000000,6147.000000,6147.000000
mean,5299.962095,2.816519,0.461500,0.211517,0.090959,0.049944,1.733125,-0.080545,0.0,0.056242,0.019034,0.233406,0.310668,0.740524,0.661949,29.741500,5479.190337,178.338864,4.97267,0.175356,0.098687,0.516837
std,3048.978645,9.274411,0.202153,0.146542,0.094888,0.073353,4.838417,0.274019,0.0,0.040914,0.136654,0.051642,2.016634,0.438383,0.473085,19.917934,7920.676234,181.028707,3.21897,0.181658,0.069444,0.499757
min,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0,0.000000,0.000000,0.063777,-1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000
25%,2671.500000,1.586437,0.333333,0.102804,0.021277,0.000000,0.442155,0.000000,0.0,0.028659,0.000000,0.201038,0.031558,0.000000,0.000000,14.000000,1596.000000,82.000000,3.00000,0.044885,0.068649,0.000000
50%,5312.000000,2.082090,0.476190,0.200000,0.067633,0.023392,0.481752,0.000000,0.0,0.046252,0.000000,0.229863,0.088608,1.000000,1.000000,25.000000,3518.000000,139.000000,5.00000,0.117318,0.088710,1.000000
75%,7949.500000,2.635978,0.607310,0.294992,0.133213,0.066667,0.560648,0.000000,0.0,0.073035,0.000000,0.259848,0.244302,1.000000,1.000000,42.000000,6314.500000,222.000000,7.00000,0.242291,0.111111,1.000000
max,10566.000000,363.000000,1.000000,1.000000,0.980392,0.980392,21.000000,0.250000,0.0,0.444444,1.000000,0.716883,113.333333,1.000000,1.000000,100.000000,180020.000000,4997.000000,22.00000,1.000000,1.000000,1.000000


In [117]:
good_news[['news_front_page', 'label']]

,news_front_page,label
0,0,0
1,0,1
2,0,1
3,0,1
4,0,0
6,0,1
8,0,1
10,0,0
11,0,0
12,0,1


In [118]:
good_news.dtypes

url                                object
urlid                               int64
boilerplate                        object
alchemy_category                   object
alchemy_category_score             object
avglinksize                       float64
commonlinkratio_1                 float64
commonlinkratio_2                 float64
commonlinkratio_3                 float64
commonlinkratio_4                 float64
compression_ratio                 float64
embed_ratio                       float64
framebased                          int64
frameTagRatio                     float64
hasDomainLink                       int64
html_ratio                        float64
image_ratio                       float64
is_news                           float64
lengthyLinkDomain                   int64
linkwordscore                       int64
news_front_page                    object
non_markup_alphanum_characters      int64
numberOfLinks                       int64
numwords_in_url                   

In [119]:
good_news.news_front_page.astype(int)
good_news['news_front_page'] = good_news.news_front_page.astype(int)

/Users/ugp/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [120]:
good_news.dtypes

url                                object
urlid                               int64
boilerplate                        object
alchemy_category                   object
alchemy_category_score             object
avglinksize                       float64
commonlinkratio_1                 float64
commonlinkratio_2                 float64
commonlinkratio_3                 float64
commonlinkratio_4                 float64
compression_ratio                 float64
embed_ratio                       float64
framebased                          int64
frameTagRatio                     float64
hasDomainLink                       int64
html_ratio                        float64
image_ratio                       float64
is_news                           float64
lengthyLinkDomain                   int64
linkwordscore                       int64
news_front_page                     int64
non_markup_alphanum_characters      int64
numberOfLinks                       int64
numwords_in_url                   

In [121]:
good_news[['news_front_page', 'label']].describe()

,news_front_page,label
count,6147.000000,6147.000000
mean,0.047828,0.516837
std,0.213420,0.499757
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,1.000000
75%,0.000000,1.000000
max,1.000000,1.000000


In [122]:
good_news['news_front_page'].isnull().any()

False

In [123]:
good_news['label'].isnull().any()

False

In [124]:
logit = LogisticRegression()

In [141]:
y = good_news['news_front_page']
x = good_news['label'].reshape(-1, 1)

In [142]:
logit.fit(x, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [143]:
logit.score(x, y)

0.9521717911176184

In [144]:
predictions = logit.predict(x)
predictions[0:5]

array([0, 0, 0, 0, 0])

In [145]:
predict_proba = logit.predict_proba(x)
predict_proba[0:5]

array([[ 0.94657865,  0.05342135],
       [ 0.95649487,  0.04350513],
       [ 0.95649487,  0.04350513],
       [ 0.95649487,  0.04350513],
       [ 0.94657865,  0.05342135]])

#### Does the website category effect green-ness?

In [146]:
data[['alchemy_category']]

,alchemy_category
0,business
1,recreation
2,health
3,health
4,sports
5,?
6,arts_entertainment
7,?
8,?
9,?


In [147]:
#muther fucking data has ?. I want to get rid of these
# ?str.replace(old, new[, max])
# data.news_front_page.str.replace('?',"Nan").astype(float)

good_alchemy = data[(data['alchemy_category'] != "?")]
good_alchemy.describe()

,urlid,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,is_news,lengthyLinkDomain,linkwordscore,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
count,5053.000000,5053.000000,5053.000000,5053.000000,5053.000000,5053.000000,5053.000000,5053.000000,5053.0,5053.000000,5053.000000,5053.000000,5053.000000,5053.000000,5053.000000,5053.000000,5053.000000,5053.000000,5053.000000,5053.000000,5053.000000,5053.000000
mean,5312.022759,2.941459,0.463598,0.213599,0.091537,0.049556,1.380019,-0.060236,0.0,0.057197,0.018999,0.234625,0.292137,0.705719,0.672670,30.067485,5438.513556,178.760538,5.056006,0.171011,0.097238,0.518504
std,3061.655693,10.136599,0.198788,0.144082,0.093382,0.071398,4.116340,0.239526,0.0,0.041449,0.136533,0.051518,1.972679,0.455764,0.469285,20.099013,7027.581558,154.565086,3.267839,0.176406,0.060423,0.499707
min,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0,0.000000,0.000000,0.070094,-1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2658.000000,1.620690,0.343434,0.108108,0.022989,0.000000,0.440870,0.000000,0.0,0.029579,0.000000,0.202473,0.032358,0.000000,0.000000,14.000000,1601.000000,82.000000,3.000000,0.044025,0.069565,0.000000
50%,5307.000000,2.107692,0.476190,0.202312,0.069444,0.024453,0.479335,0.000000,0.0,0.046414,0.000000,0.230635,0.088136,1.000000,1.000000,26.000000,3623.000000,141.000000,5.000000,0.116505,0.089094,1.000000
75%,7983.000000,2.682927,0.607029,0.296552,0.132743,0.066667,0.547486,0.000000,0.0,0.074398,0.000000,0.260788,0.240000,1.000000,1.000000,43.000000,6413.000000,228.000000,7.000000,0.237726,0.110577,1.000000
max,10566.000000,363.000000,1.000000,1.000000,0.980392,0.980392,21.000000,0.214286,0.0,0.444444,1.000000,0.716883,113.333333,1.000000,1.000000,100.000000,87948.000000,3283.000000,21.000000,1.000000,1.000000,1.000000


In [148]:
good_alchemy[['alchemy_category', 'label']]

,alchemy_category,label
0,business,0
1,recreation,1
2,health,1
3,health,1
4,sports,0
6,arts_entertainment,1
10,business,0
11,sports,0
12,health,1
14,recreation,0


In [149]:
good_alchemy.dtypes

url                                object
urlid                               int64
boilerplate                        object
alchemy_category                   object
alchemy_category_score             object
avglinksize                       float64
commonlinkratio_1                 float64
commonlinkratio_2                 float64
commonlinkratio_3                 float64
commonlinkratio_4                 float64
compression_ratio                 float64
embed_ratio                       float64
framebased                          int64
frameTagRatio                     float64
hasDomainLink                       int64
html_ratio                        float64
image_ratio                       float64
is_news                           float64
lengthyLinkDomain                   int64
linkwordscore                       int64
news_front_page                    object
non_markup_alphanum_characters      int64
numberOfLinks                       int64
numwords_in_url                   

urlid  avglinksize  commonlinkratio_1  commonlinkratio_2  \
0      4042     2.055556           0.676471           0.205882   
1      8471     3.677966           0.508021           0.288770   
2      1164     2.382883           0.562016           0.321705   
3      6684     1.543103           0.400000           0.100000   
4      9006     2.676471           0.500000           0.222222   
6      8685     0.773810           0.215054           0.053763   
10     2293     2.506527           0.637755           0.293367   
11     3651     4.986111           0.640000           0.426667   
12     1917     3.056911           0.595588           0.227941   
14     2110     2.299492           0.547414           0.206897   
15     5318     0.990431           0.522523           0.108108   
16     9842     0.929825           0.068966           0.000000   
17     3946     2.083333           0.421053           0.178947   
18      334     3.164384           0.586207           0.275862   
19      271     2.826772           0.669065           0.237410   
20     6984     4.103627           0.515556           0.297778   
21     8979     2.033333           0.516129           0.172043   
22     1502     1.333333           0.000000           0.000000   
25     1499     4.800000           0.603448           0.379310   
26     7298     3.355263           0.488372           0.290698   
27     3294     3.321429           0.505747           0.298851   
30     1391     1.985816           0.356643           0.139860   
31     2152     1.571429           0.451327           0.168142   
33     2807     2.166667           0.447514           0.207182   
34      497     2.544041           0.583333           0.324074   
37     1650     1.549451           0.417910           0.114428   
39    10154     2.188312           0.569061           0.149171   
40     7461     9.800000           0.292308           0.092308   
41     4064     2.086957           0.498195           0.202166   
43      383     1.783333           0.214286           0.071429   
...     ...          ...                ...                ...   
7351   2912     1.200000           0.091667           0.008333   
7352   9775     2.198630           0.693878           0.340136   
7353   6122     3.627907           0.987597           0.973643   
7354   9149     1.965174           0.556604           0.226415   
7355   9277     1.840000           0.484848           0.060606   
7358   7480     4.993827           0.802198           0.670330   
7359     96     2.260000           0.448276           0.068966   
7360   7532     2.097561           0.370079           0.133858   
7362   5211     1.765432           0.602410           0.132530   
7364   8891     1.476190           0.366197           0.098592   
7370   8563     1.677966           0.256757           0.135135   
7371   9192     1.219178           0.240000           0.066667   
7372   3566     1.168367           0.627551           0.301020   
7373   3788     2.085774           0.747390           0.488518   
7374   7485     2.093333           0.349057           0.150943   
7375   7425     1.978495           0.510638           0.308511   
7376   4572     7.392157           0.743421           0.460526   
7377  10509     1.861789           0.347222           0.104167   
7378   5183     2.488753           0.791583           0.509018   
7379   7696     2.373134           0.375000           0.277778   
7381   4475     1.000000           0.000000           0.000000   
7382   8327     5.611570           0.738318           0.579439   
7383  10126     1.687500           0.531250           0.062500   
7386    546     3.156863           0.669565           0.313043   
7387  10382     1.753731           0.510791           0.251799   
7388   7634     1.995098           0.539535           0.213953   
7390   8958     3.010526           0.474747           0.222222   
7391   8895     2.208054           0.483333           0.246667   
7392   1191     2.000000           0.315789   

In [23]:
# alchemy_category vs label

# Absolutely, The more frequently updated the website (and i would assume a news site would be frequently updated)
# dyncmically the less "brown" a website would be.

# news_front_page vs label

import statsmodels.api as sm

y1 = a_data['alchemy_category']
x1 = a_data['label']

# print x
# print y


logit = sm.Logit(y1, x1) #set up a STATSMODELS Logistic Regression
result = logit.fit() # FIT your Logistic Regression to your data

print result.summary() # See the results of your Logistic Regression
predictions = result.predict(x1) # Get predictions for x's

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

#### Does the image ratio effect green-ness?

In [155]:
# image_ratio vs label

import statsmodels.api as sm

y = data['image_ratio']
x = data['label'].reshape(-1, 1)

x
# y = good_news['news_front_page']
# x = good_news['label'].reshape(-1, 1)

# logit = sm.Logit(y, x) #set up a STATSMODELS Logistic Regression
# result = logit.fit() # FIT your Logistic Regression to your data

# print result.summary() # See the results of your Logistic Regression
# predictions = result.predict(x) # Get predictions for x's

array([[0],
       [1],
       [1],
       ..., 
       [1],
       [1],
       [0]])

#### Fit a logistic regression model using statsmodels
- Test different features that may be valuable
- Examine the coefficients, does the feature increase or decrease the effect of being evergreen?

In [7]:
# TODO

#### Fit a logistic regression model using statsmodels with text features
- Add text features that may be useful, add this to the model and see if they improve the fit
- Examine the coefficients, does the feature increase or decrease the effect of being evergreen?

In [156]:
# EXAMPLE text feature 'recipe'
data['is_recipe'] = data['title'].fillna('').str.contains('recipe')